#**6 LOS Upper Fence Text Only**

## **6.1 Setup**

In [ ]:
# First install package from terminal:
!pip install autogluon  # autogluon==0.4.1
!pip install lime
!pip install torchaudio -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.1/229.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.0/306.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.7/416.7 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.5/120.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283853 sha256=925bc3bf3d4c719e297496af1ac8088b97c327145d6f1f6af1c2ca13f526b6a8
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [ ]:
# import all required modules
import pandas as pd
import numpy as np
import os
import re
import random
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', None)  ###
pd.set_option('display.max_columns', None)  ###
pd.set_option('display.width', None)  ###
pd.set_option('display.max_colwidth', None)  ###

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report
from sklearn.metrics import auc
from sklearn.metrics import cohen_kappa_score
from sklearn.inspection import permutation_importance
#from autogluon.tabular import TabularPredictor
#from autogluon.multimodal import MultiModalPredictor
#from autogluon.tabular import TabularPredictor as ModalPredictor

import torch
from transformers import BertTokenizerFast, BertModel, BertConfig, BertForPreTraining, BertForSequenceClassification, pipeline
#import lime
#import lime.lime_text
import pickle

In [1]:
try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

if IN_COLAB:
  print("We're running Colab")

if IN_COLAB:
  # Mount the Google Drive at mount
  mount='/content/gdrive'
  print("Colab: mounting Google drive on ", mount)
  # connect your colab with the drive
  drive.mount(mount)

 # Switch to the directory on the Google Drive that you want to use
  import os
  path_to_repo = mount + "/MyDrive/MIMIC Project/"

else:
  # Setup Repository
  with open("repo_info.txt", "r") as repo_info:
      path_to_repo = repo_info.readline()


print(path_to_repo)

path_to_data = f"{path_to_repo}data/"
# path_to_raw = f"{path_to_data}raw/"
path_to_processed = f"{path_to_data}processed/"
path_to_lda = f"{path_to_data}lda/"
# path_to_icd = f"{path_to_data}icd_codes/"
path_to_models = f"{path_to_repo}models/"
path_to_results = f"{path_to_repo}results/"
# path_to_tables = f"{path_to_repo}tables"

os.makedirs(path_to_processed, exist_ok = True)

We're running Colab
Colab: mounting Google drive on  /content/gdrive
Mounted at /content/gdrive
/content/gdrive/MyDrive/MIMIC Project/


## **6.2 Import the Dataset**

In [ ]:
def load_datasets(method):
    """
    Function to load train, test and validation set based on the chosen method
    method: string for the processing method we want to load
    """
    global path_to_processed
    # load it back
    train = pd.read_feather('{}{}train_{}{}{}{}'.format(path_to_processed, method, seed_tag, preproc_tag, lemma_tag, death_tag))
    test = pd.read_feather('{}{}test_{}{}{}{}'.format(path_to_processed, method, seed_tag, preproc_tag, lemma_tag, death_tag))
    return train, test

In [ ]:
# compute other metrics
def perf_evaluator(y_test, y_pred, y_pred_proba):
    """ Function to display the main classification performance metrics """
    kappa = cohen_kappa_score(y_test, y_pred)
    precision, recall, prc_th = precision_recall_curve(y_test, y_pred_proba)
    prc_auc = auc(recall, precision)
    return kappa, prc_auc

In [ ]:
# Model Parameters
label = 'los_cat_UF'
metric = 'roc_auc'

# Set to True if we want to include deaths
death_incl = False
death_tag = np.where(death_incl,"_death", "")

# Iterate over our main methods of vectorization

vect_dict = {'stemming': (False, False),
             'spacy': (True, True)}


for key, value in vect_dict.items():
    print(key)
    # PARAMETERS

    lemmatize = value[0] # set to false if we want to do stemming
    lemma_tag = str(np.where(lemmatize, "_lemma",""))
    spacy = value[1]
    if spacy: lemma_tag = str(np.where(lemmatize, "_lemma_spacy",""))

    preprocessing = True # set to true if we want to clean and perform some preprocessing
    preproc_heavier = True # set to True if we want a heavier preprocessing
    preproc_tag_2 = np.where(preproc_heavier, '_heavier', '')
    preproc_tag = np.where(preprocessing, f'_preproc{preproc_tag_2}', f'{preproc_tag_2}')

    df = pd.read_feather(f'{path_to_processed}df_los_UF_{preproc_tag}{lemma_tag}{death_tag}') # 1st change: df_los --->df_los_80_
    # restrict to just discharge notes
    df = df[['los_cat_UF', 'text']]                                                               # 2nd change: 'los_cat' ---> 'los_cat_80'
    print('Dataframe Loaded')
    # split the data into training and test
    train, test = train_test_split(df, train_size=0.80, stratify = df['los_cat_UF'], random_state=42) # 3rd change: los_cat --->los_cat_80

    # run the tabular predictor ensemble of models (include 'multimodal' hyperparameter)
    save_path = f'{path_to_models}text_los_UF_{preproc_tag}{lemma_tag}{death_tag}'                            # 4th change: text--->text_los_80
    os.makedirs(save_path, exist_ok = True)


    try:
        # Attempt to load the model
        predictor = MultiModalPredictor.load(save_path, resume=True)
        print("Model Loaded")
    except FileNotFoundError:
        print("Model not found. Training a new one...")
        # Train a new model
        predictor = MultiModalPredictor(label=label, eval_metric=metric, path=save_path)
        predictor.fit(
            train_data=train,
            hyperparameters={
                "model.hf_text.checkpoint_name": "emilyalsentzer/Bio_ClinicalBERT",
            },
        )


    # evaluate performance on the test set
    per_tab = predictor.evaluate(test, metrics=['roc_auc','f1','acc'])
    print(f"\nTest set performance:\n{per_tab}")
    # save the class and probability predictions
    y_pred = predictor.predict(test)
    y_pred_proba = predictor.predict_proba(test).iloc[:,1]
    perf = perf_evaluator(test['los_cat_UF'], y_pred, y_pred_proba)                                  # 5th change:los_cat--->los_cat_80
    perf_dict = {"Cohen's Kappa": perf[0], "PRC AUC": perf[1]}
    print(f"\nPerformance metrics:\n{perf_dict}")
    perf_dict.update(per_tab)

    # save performances
    df_perf = pd.DataFrame.from_dict(perf_dict, orient='index', columns=['performances'])
    df_perf.to_excel(path_to_results+f'text_only/df_perf_los_UF_{key}{death_tag}.xlsx')           #6th change:df_perf_ --->df_perf_los_80_

stemming


=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sat Nov 18 15:31:17 UTC 2023
CPU Count:          8
Pytorch Version:    2.2.1+cu121
CUDA Version:       12.1
Memory Avail:       45.34 GB / 50.99 GB (88.9%)
Disk Space Avail:   52.74 GB / 200.00 GB (26.4%)
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [False, True]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])


Dataframe Loaded
Model not found. Training a new one...



AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir /content/gdrive/MyDrive/MIMIC_IV/models/text_los_UF__preproc_heavier
    ```

INFO: Global seed set to 0


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

...

Predicting: 0it [00:00, ?it/s]


Test set performance:
{'roc_auc': 0.8415046409500623, 'f1': 0.2643856920684292, 'acc': 0.931667148223057}


Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]


Performance metrics:
{"Cohen's Kappa": 0.2403476932514711, 'PRC AUC': 0.3736276385717823}


## **6.3  LIME Explainability**

In [ ]:
class AutogluonWrapper:
    """ Wrapper function to extract probability predictions from our model"""
    def __init__(self, predictor, feature_names):
        self.ag_model = predictor
        self.feature_names = feature_names

    def predict_binary_prob(self, X):
        if isinstance(X, pd.Series):
            X = X.values.reshape(1,-1)
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X, columns=self.feature_names)
        prob = self.ag_model.predict_proba(X, as_multiclass=True)
        return np.array(prob)

In [ ]:
def return_weights(exp):

    """Get weights from LIME explanation object"""

    exp_list = exp.as_list()
    exp_weight = [x[1] for x in exp_list]
    exp_labels = [x[0] for x in exp_list]

    return exp_weight, exp_labels

In [ ]:
# Model Parameters
label = 'los_cat_UF'
metric = 'roc_auc'

# Set to True if we want to include deaths
death_incl = False
death_tag = np.where(death_incl,"_death", "")

# Iterate over our main methods of vectorization

vect_dict = {
    # 'stemming': (False, False),
             'spacy': (True, True)}


for key, value in vect_dict.items():
    print(key)
    # PARAMETERS

    lemmatize = value[0] # set to false if we want to do stemming
    lemma_tag = str(np.where(lemmatize, "_lemma",""))
    spacy = value[1]
    if spacy: lemma_tag = str(np.where(lemmatize, "_lemma_spacy",""))

    preprocessing = True # set to true if we want to clean and perform some preprocessing
    preproc_heavier = True # set to True if we want a heavier preprocessing
    preproc_tag_2 = np.where(preproc_heavier, '_heavier', '')
    preproc_tag = np.where(preprocessing, f'_preproc{preproc_tag_2}', f'{preproc_tag_2}')




    print('Dataframe Loaded')
    df = pd.read_feather(f'{path_to_processed}df_los_UF_{preproc_tag}{lemma_tag}{death_tag}') # 1st change: df_los --->df_los_80_
    # restrict to just discharge notes
    df = df[['los_cat_UF', 'text']]



    # split the data into training and test


    train, test = train_test_split(df, train_size=0.80, stratify = df['los_cat_UF'], random_state=42)                               # 1st change

    # run the tabular predictor ensemble of models (include 'multimodal' hyperparameter)
    save_path = f'{path_to_models}text_los_UF_{preproc_tag}{lemma_tag}{death_tag}'

    # Reload our predictor model
    predictor = MultiModalPredictor.load(save_path)

    # Define the path where we have saved various parts of our model

    tokenizer_path = f'{save_path}/hf_text/'
    bert_path = f'{save_path}/hf_text/config.json'
    checkpoint_path = f'{save_path}/model.ckpt'

    tokenizer = BertTokenizerFast.from_pretrained(tokenizer_path)

    # Define the LIME Explainer object
    lime_explainer = lime.lime_text.LimeTextExplainer(class_names = ['short', 'long'], random_state = 42)

    # Initialize our AutoGluon Wrapper
    autogluon_wrap = AutogluonWrapper(predictor, ['text'])

    # Print an example to check if everything works
    sample = lime_explainer.explain_instance(test.text.iloc[0], autogluon_wrap.predict_binary_prob, num_samples = 100)
    sample.show_in_notebook()

    # load weights
    try :
        with open(f'{path_to_data}feature_importance/text_only_weights_los_UF_{preproc_tag}{lemma_tag}{death_tag}.pkl', 'rb') as handle:       # 2nd change
            weights = pickle.load(handle)
            print('Weights loaded')
            print(len(weights))
    except :
        weights = [] # initialize an empty dictionary if no existing file is present
        print('New Weight List')
    # load labels
    try :
        with open(f'{path_to_data}feature_importance/text_only_labels_los_UF_{preproc_tag}{lemma_tag}{death_tag}.pkl', 'rb') as handle:        # 3rd change
            labels = pickle.load(handle)
            print('Label Loaded')
            print(len(labels))
    except :
        labels = [] # initialize an empty dictionary if no existing file is present
        print('New Label List')


    minimum = len(weights)  # beginning of our list
    maximum = len(test) # end of our list len(user_list)
    steps = 500 # how many users per batch
    range_list = list(np.arange(minimum,maximum,steps))

    iter_count = 0

    if maximum != len(weights):
      for i in range(len(range_list)):
        min_r = range_list[i]
        try:
          max_r = range_list[i+1]
        except:
          max_r = maximum
        print("Range: {} to {}".format(min_r, max_r))
        batch_pred = test.iloc[min_r:max_r]
        #Iterate over first 100 rows in feature matrix
        for index, row in batch_pred.iterrows():
            input_ids = tokenizer.encode_plus(row.text, add_special_tokens = False, return_tensors = 'pt')
            input_ids = list(input_ids['input_ids'][0].split(512-2))[0]
            row.text = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids))
            #Get explanation
            exp = lime_explainer.explain_instance(row.text,
                                        autogluon_wrap.predict_binary_prob,
                                        num_features=100,
                                        num_samples = 500)
            #Get weights
            exp_weight, exp_labels = return_weights(exp)
            weights.append(exp_weight)
            labels.append(exp_labels)
        iter_count += steps
        if iter_count % 500 == 0 or max_r != maximum:
          # save the file
          with open(f'{path_to_data}feature_importance/text_only_weights_los_UF_{preproc_tag}{lemma_tag}{death_tag}.pkl', 'wb') as f:              # 4th
              pickle.dump(weights, f)
          with open(f'{path_to_data}feature_importance/text_only_labels_los_UF_{preproc_tag}{lemma_tag}{death_tag}.pkl', 'wb') as f:                     # 5th
              pickle.dump(labels, f)
          print("Output Saved")


    # Now we process the list of labels and weights
    words = [word for single_list in labels for word in single_list]
    print(len(words))
    words = list(set(words)) # get unique values
    len(words)
    # Get a dictionary of words
    dict_of_words = {i:0 for i in words}

    # And get the correct weights
    for count, single_list in enumerate(weights):
      for i, weight in enumerate(single_list):
        dict_of_words[labels[count][i]] += weight

    # Transform everything to a pandas dataframe
    weight_df = pd.DataFrame([dict_of_words.keys(), dict_of_words.values()]).T
    weight_df.columns = ['word', 'weight']


    # sort values by absolute mean
    abs_mean = weight_df.sort_values('weight', ascending = True).head(30)
    abs_mean.weight = abs_mean.weight/len(weights)
    # normalize by 100%
    abs_mean.weight = abs_mean.weight/abs_mean.weight.max()*100

    #Plot abs mean --------------------------------------------------
    fig, ax = plt.subplots(nrows=1, ncols=1,figsize=(20,15))

    y_ticks = range(len(abs_mean))
    y_labels = abs_mean.word
    plt.barh(y=y_ticks,width=abs_mean.weight)

    plt.yticks(ticks=y_ticks,labels=y_labels,size= 15)
    plt.title('LIME Weights - Average Absolute Values of Test Set Explanations')
    plt.ylabel('')
    plt.xlabel('Mean |Weight|',size=20)

    plt.savefig(f"{path_to_repo}results/text_only/lime_los_UF_{preproc_tag}{lemma_tag}{death_tag}", bbox_inches='tight')    # 7th
    plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
!pip install textacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 77.3 MB/s eta 0:00:00


In [ ]:
df = pd.read_feather(f'{path_to_processed}df_mortality__preproc_heavier')


In [ ]:
text=df['text']

In [ ]:
import textacy
from textacy.extract.kwic import keyword_in_context
import random
import re
import pandas as pd

# Assuming 'test' is your test DataFrame from the provided code.
# And 'path_to_data' is your variable to the directory containing the data.

def KWIC(text, keyword, window=35):
    return list(keyword_in_context(text, keyword, ignore_case=True, window_width=window))

def kwic(doc_series, keyword, window=35, print_samples=5):
    """
    Generate Key Word In Context (KWIC) for a given keyword and document series.

    Args:
        doc_series (pd.Series): A Pandas Series containing text documents.
        keyword (str): The keyword to search for within the documents.
        window (int): The number of characters to show around the keyword. Default is 35.
        print_samples (int): The number of KWIC samples to print. Default is 5.

    Returns:
        A list of KWIC results or None if printing only.
    """
    kwic_list = []
    for text in doc_series:
        kwic_list.extend(KWIC(text, keyword, window))

    if print_samples is None or print_samples == 0:
        return kwic_list
    else:
        k = min(print_samples, len(kwic_list))
        print(f"{k} random samples out of {len(kwic_list)} contexts for '{keyword}':")
        for sample in random.sample(kwic_list, k):
            left, keyword, right = sample
            print(re.sub(r'[\n\t]', ' ', left) + '  ' + keyword + '  ' + re.sub(r'[\n\t]', ' ', right))


keyword_to_analyze = 'fever'

kwic_samples = kwic(text, keyword=keyword_to_analyze, window=35, print_samples=10)



10 random samples out of 51236 contexts for 'fever':
 iv fluid antibiot patient continu   fever   treatment blood urin sputum cultur
 weak numb unabl toler food liquid   fever   1004 chill short breath chest pain
l fatigu weak fever 99 per pt deni   fever   chill chest pain short breath abdo
rat beta blocker throughout admiss   fever   cx sent sever time multipl sourc g
 2 tablet po q4h everi 4 hour need   fever  pain 5 magnesium hydroxid 400 mg5 m
inu one week transamin initi start   fever   leukocytosi patient also develop t
codnari acut alcohol hepat multipl   fever   workup negat medic admiss lactulos
ibl experi chest pain short breath   fever   1004 weight gain shaki lightheaded
it stabl discharg instruct develop   fever   1015 chill sweat nausea vomit wors
consult rec fever manag id suggest   fever   like secondari periop reason ie en
